In [2]:
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler

# === เตรียมข้อมูล Iris ===
X, y = load_iris(return_X_y=True)
X = X[y == 0][:50]  # ใช้เฉพาะ class Setosa
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# === VQC: 16 qubits → 4 features (4 bits/feature) ===
n_qubits = 16
params = np.random.uniform(0, 2 * np.pi, size=n_qubits)

qc = QuantumCircuit(n_qubits)
for i in range(n_qubits):
    qc.ry(params[i], i)
for i in range(n_qubits - 1):
    qc.cx(i, i + 1)

qc.measure_all()

# === Run circuit ===
backend = Aer.get_backend("qasm_simulator")
shots = 2048
tqc = transpile(qc, backend)
job = backend.run(tqc, shots=shots)
counts = job.result().get_counts()

# === Decode: 4 bits per feature → 4 features ===
def bits_to_float(bits):
    return int(bits, 2) / 15.0  # 0–15 mapped to 0.0–1.0

def bitstring_to_features(bs):
    bs = bs[::-1]  # Qiskit returns reversed bitstring
    return [bits_to_float(bs[i:i+4]) for i in range(0, 16, 4)]

features = []
for bs, freq in counts.items():
    decoded = bitstring_to_features(bs)
    features.extend([decoded] * freq)

features = np.array(features)

# === กลับไปเป็นข้อมูล Iris จริง ===
generated = scaler.inverse_transform(features)
df = pd.DataFrame(generated, columns=["sepal length", "sepal width", "petal length", "petal width"])
df["label"] = "Generated (16-qubit VQC)"
print(df.head())

# === save ===
df.to_csv("6_generated_iris_16qubit.csv", index=False)


   sepal length  sepal width  petal length  petal width  \
0           5.6         4.12          1.00     0.466667   
1           5.7         3.14          1.48     0.466667   
2           4.4         2.58          1.48     0.266667   
3           4.4         2.58          1.48     0.266667   
4           4.4         2.58          1.48     0.266667   

                      label  
0  Generated (16-qubit VQC)  
1  Generated (16-qubit VQC)  
2  Generated (16-qubit VQC)  
3  Generated (16-qubit VQC)  
4  Generated (16-qubit VQC)  
